# Python и машинное обучение: нейронные сети и компьютерное зрение

## Модуль 5. Распознавание объектов на изображениях



### YOLOv5

По состоянию на декабрь 2023 года - "базовая" и самая простая модель для поиска объектов на фотографиях. Загружаем на ```torch.hub```.


In [ ]:
!pip install -U ultralytics

In [ ]:
import torch
import cv2
from PIL import Image

from torchinfo import summary

import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else \
    "mps" if torch.backends.mps.is_built() else "cpu"
device

%matplotlib inline

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True)

print(model)

In [ ]:
!wget https://ultralytics.com/images/zidane.jpg -q

In [ ]:
img = cv2.imread('zidane.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
display(Image.fromarray(img))

In [ ]:
results = model('zidane.jpg')

df = results.pandas().xyxy[0]
df

In [ ]:
results.print()
results.xyxy[0]  # img1 predictions (tensor)

In [ ]:
# tl = round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1
tl=2
tf = max(tl - 1, 1)


for i, row in df.iterrows():
    xA, yA, xB, yB = map(int, row[:4])
    cv2.rectangle(img, (xA, yA), (xB, yB), (0, 255, 0), tl)
    cv2.putText(img, row['name'], (xA, yA - 2), 0, tl / 3, [0, 255, 0], thickness=tf, lineType=cv2.LINE_AA) 
    
display(Image.fromarray(img))

## Дообучение модели YOLO

https://colab.research.google.com/drive/1Plz91PHWwf04bYt21mnWp7qBcDIvJ6J6

https://medium.com/mlearning-ai/training-yolov5-custom-dataset-with-ease-e4f6272148ad


### Detectron2

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = (8.0, 8.0)

%matplotlib inline

In [ ]:
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
im = cv2.imread("./input.jpg")
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
plt.imshow(im)
plt.show()

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
mask = outputs['instances'].pred_masks.cpu().numpy().astype('uint8')[0]
contour, _ = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

image_with_overlaid_predictions = im.copy()
cv2.drawContours(image_with_overlaid_predictions, [contour[0]], -1, (0,255,0), 1)

plt.figure(figsize=(14,14))
plt.imshow(image_with_overlaid_predictions)
plt.show()